# Fine-Tuning Llama 3.2-3B with Unsloth

This notebook demonstrates fine-tuning the **Llama 3.2-3B** model using **Unsloth** and the [BAAI Infinity-Instruct dataset](https://huggingface.co/datasets/BAAI/Infinity-Instruct). The dataset contains over 660,000 examples, providing a rich resource for training robust, conversational AI models.

After fine-tuning, the final model will be deployed on Hugging Face for easy access and use.

(if you want to run this i recommend using Google Colab)

In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Infinity-Instruct](https://huggingface.co/datasets/BAAI/Infinity-Instruct) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("BAAI/Infinity-Instruct", '7M_domains',split = "train", token = "hf_token" )

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/76 [00:00<?, ?it/s]

train-00000-of-00014.parquet:   0%|          | 0.00/69.7M [00:00<?, ?B/s]

train-00001-of-00014.parquet:   0%|          | 0.00/69.7M [00:00<?, ?B/s]

train-00002-of-00014.parquet:   0%|          | 0.00/70.1M [00:00<?, ?B/s]

...

train-00011-of-00014.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

train-00012-of-00014.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

train-00013-of-00014.parquet:   0%|          | 0.00/71.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/7449106 [00:00<?, ? examples/s]

Map:   0%|          | 0/7449106 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [8]:
dataset[5]["conversations"]

[{'content': "John hired a car for a family tour. The hire cost is $200. He needed to buy 10 gallons of gas at a price of $4 per gallon. There's a mileage charge of $0.55 for each mile driven. If the total mileage is 350 miles, what's the total cost incurred by John?",
  'role': 'user'},
 {'content': 'The cost of gas is 10*4=40. The cost for driving 350 miles is 0.55*350=0.55. The total cost of the trip includes the rent cost, gas cost, and the cost for miles driven. Hence the total cost is 200 + 40 + 0.55 = 432.5. The answer is 432.5.',
  'role': 'assistant'}]

In [9]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nJohn hired a car for a family tour. The hire cost is $200. He needed to buy 10 gallons of gas at a price of $4 per gallon. There's a mileage charge of $0.55 for each mile driven. If the total mileage is 350 miles, what's the total cost incurred by John?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe cost of gas is 10*4=40. The cost for driving 350 miles is 0.55*350=0.55. The total cost of the trip includes the rent cost, gas cost, and the cost for miles driven. Hence the total cost is 200 + 40 + 0.55 = 432.5. The answer is 432.5.<|eot_id|>"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Instantiate the SFTTrainer for supervised fine-tuning
trainer = SFTTrainer(
    model=model,  # The pre-trained model to fine-tune
    tokenizer=tokenizer,  # Tokenizer corresponding to the model
    train_dataset=dataset,  # Dataset containing the training data
    dataset_text_field="text",  # Field name in the dataset that contains text data
    max_seq_length=max_seq_length,  # Maximum sequence length; longer sequences will be truncated
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # Function to collate and batch data
    dataset_num_proc=2,  # Number of processes for data preprocessing (speeds up data loading)
    packing=False,  # When True, packs multiple short sequences into one (speeds up training for short sequences)
    # Can make training up to 5x faster for short sequences by reducing padding and increasing GPU utilization
    args=TrainingArguments(  # Training configuration parameters
        per_device_train_batch_size=2,  # Batch size per device (e.g., per GPU)
        gradient_accumulation_steps=4,  # Number of steps to accumulate gradients before updating model weights
        # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps * num_devices
        warmup_steps=5,  # Number of steps to linearly increase the learning rate from zero to the initial value
        # num_train_epochs=1,  # Total number of training epochs; uncomment to set epochs instead of steps
        max_steps=100,  # Total number of training steps; overrides num_train_epochs if set
        learning_rate=2e-4,  # Initial learning rate for the optimizer
        fp16=not is_bfloat16_supported(),  # Use FP16 precision if bfloat16 is not supported
        bf16=is_bfloat16_supported(),  # Use bfloat16 precision if supported (saves memory and can speed up training)
        logging_steps=1,  # Log training metrics every 'logging_steps' steps
        optim="adamw_8bit",  # Optimizer type; 'adamw_8bit' uses 8-bit quantization to reduce memory usage
        weight_decay=0.01,  # Weight decay coefficient for L2 regularization to prevent overfitting
        lr_scheduler_type="linear",  # Learning rate scheduler type; 'linear' decreases LR linearly after warmup
        seed=3407,  # Random seed for reproducibility
        output_dir="outputs",  # Directory to save model checkpoints and logs
        report_to="none",  # Platform to report training metrics ('none' disables reporting)
        # Use 'wandb' or other platforms here to enable logging to those services
    ),
)

Map (num_proc=2):   0%|          | 0/7449106 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# model.save_pretrained("Llama3.2-3b-SFF-Infinity-MateoRovere") # Local saving
# tokenizer.save_pretrained("Llama3.2-3b-SFF-Infinity-MateoRovere")
# model.push_to_hub("MateoRov/Llama3.2-3b-SFF-Infinity-MateoRovere", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

### Saving to HuggingFace.

In [ ]:
# Merge to 16bit
model.save_pretrained_merged("Llama3.2-3b-SFF-Infinity-MateoRovere-7MTraining", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("MateoRov/Llama3.2-3b-SFF-Infinity-MateoRovere-7MTraining", tokenizer, save_method = "merged_16bit", token = "hf_token")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion

Some supported quant methods:
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.


In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )